# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [36]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [37]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [38]:
import os

PRICE_DATA = os.getenv("PRICE_DATA")


In [39]:
from glob import glob

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
parquet_files


['../../05_src/data/prices\\A\\A_2000\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2001\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2002\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2003\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2004\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2005\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2006\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2007\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2008\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2009\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2010\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2011\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2012\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2013\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2014\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2015\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2016\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2017\\part.0.pa

In [40]:
# read the parquet files into a df and set Ticker as index
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")
dd_px

,Date,Adj Close,Close,High,Low,Open,Volume,Year
npartitions=13078,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [41]:
# Add lags for variables Close and Adj_Close.
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1)
                       ,Adj_Close_lag_1 = x['Adj Close'].shift(1))
   )

dd_shift


C:\Users\tinti\AppData\Local\Temp\ipykernel_16572\1944904817.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1
npartitions=13078,,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [42]:
#Add returns based on Close:

dd_rets = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1']
)

dd_rets


,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns
npartitions=13078,,,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...


In [43]:
# Add range
dd_range = dd_rets.assign(
    hi_lo_range = lambda x: x['High']-x['Low'] 
)

dd_range

,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range
npartitions=13078,,,,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
# testing / understanding  if we can use dask to calculating rolling mean (please ignore it for assignment purpose)

dd_test = dd_range.assign(: x['Returns'].rolling(10).mean())
dd_test

ValueError: Can only rolling dataframes with known divisions
See https://docs.dask.org/en/latest/dataframe-design.html#partitions
for more information.

In [29]:
# Assign result to dd_feat and covert the Dask data frame to pandas data frame.
dd_feat = dd_range.compute()
dd_feat


c:\Users\tinti\miniconda3\envs\dsi_participant\lib\site-packages\pyarrow\pandas_compat.py:766: DeprecationWarning: DatetimeTZBlock is deprecated and will be removed in a future version. Use public APIs instead.
  klass=_int.DatetimeTZBlock,
c:\Users\tinti\miniconda3\envs\dsi_participant\lib\site-packages\pandas\core\frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range
Ticker,,,,,,,,,,,,
DOV,2025-01-02 00:00:00+00:00,185.639999,185.639999,189.000000,184.929993,188.320007,651800.0,2025,NaN,NaN,NaN,4.070007
DOV,2025-01-03 00:00:00+00:00,188.029999,188.029999,188.470001,185.160004,186.660004,404500.0,2025,185.639999,185.639999,0.012874,3.309998
DOV,2025-01-06 00:00:00+00:00,188.029999,188.029999,190.300003,187.479996,188.000000,453600.0,2025,188.029999,188.029999,0.000000,2.820007
DOV,2025-01-07 00:00:00+00:00,187.979996,187.979996,189.199997,186.869995,187.910004,692500.0,2025,188.029999,188.029999,-0.000266,2.330002
DOV,2025-01-08 00:00:00+00:00,187.220001,187.220001,187.850006,185.509995,187.009995,653200.0,2025,187.979996,187.979996,-0.004043,2.340012
...,...,...,...,...,...,...,...,...,...,...,...,...
CTLT,2020-12-24 00:00:00+00:00,105.309998,105.309998,106.220001,104.660004,104.889999,277767.0,2020,104.860001,104.860001,0.004291,1.559998
CTLT,2020-12-28 00:00:00+00:00,102.879997,102.879997,106.550003,102.779999,106.139999,582003.0,2020,105.309998,105.309998,-0.023075,3.770004
CTLT,2020-12-29 00:00:00+00:00,102.709999,102.709999,103.644997,101.180000,103.370003,549151.0,2020,102.879997,102.879997,-0.001652,2.464996


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [32]:
# Add a new feature containing the moving average of returns using a window of 10 days.

dd_feat = dd_feat.assign(
    moving_avg = lambda x: x['Returns'].rolling(10).mean())

dd_feat



Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range,moving_avg
Ticker,,,,,,,,,,,,,
DOV,2025-01-02 00:00:00+00:00,185.639999,185.639999,189.000000,184.929993,188.320007,651800.0,2025,NaN,NaN,NaN,4.070007,NaN
DOV,2025-01-03 00:00:00+00:00,188.029999,188.029999,188.470001,185.160004,186.660004,404500.0,2025,185.639999,185.639999,0.012874,3.309998,NaN
DOV,2025-01-06 00:00:00+00:00,188.029999,188.029999,190.300003,187.479996,188.000000,453600.0,2025,188.029999,188.029999,0.000000,2.820007,NaN
DOV,2025-01-07 00:00:00+00:00,187.979996,187.979996,189.199997,186.869995,187.910004,692500.0,2025,188.029999,188.029999,-0.000266,2.330002,NaN
DOV,2025-01-08 00:00:00+00:00,187.220001,187.220001,187.850006,185.509995,187.009995,653200.0,2025,187.979996,187.979996,-0.004043,2.340012,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
CTLT,2020-12-24 00:00:00+00:00,105.309998,105.309998,106.220001,104.660004,104.889999,277767.0,2020,104.860001,104.860001,0.004291,1.559998,0.010260
CTLT,2020-12-28 00:00:00+00:00,102.879997,102.879997,106.550003,102.779999,106.139999,582003.0,2020,105.309998,105.309998,-0.023075,3.770004,0.007900
CTLT,2020-12-29 00:00:00+00:00,102.709999,102.709999,103.644997,101.180000,103.370003,549151.0,2020,102.879997,102.879997,-0.001652,2.464996,0.004899


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Yes, it is necessary to convert to pandas to calculate the moving average return. 
It would not be better if using Dask. This is because Dask is better then Pandas in speed becasue it enables parallel computing in python. However , for "rolling 10 day average", it takes into account of the previous row values which can't be compute parallelly base on the current partition. Therefore it has no advantage over Pandas and can't be computed in this case.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.